In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM,BitsAndBytesConfig
import torch
from datasets import load_dataset
import time

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model_name = "EleutherAI/gpt-j-6B"
# model_name = "tiiuae/falcon-7b"
# model_name = "mosaicml/mpt-7b"
# model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "NousResearch/Nous-Hermes-llama-2-7b"
# model_name = "databricks/dolly-v2-12b"  # right on the memory edge
# model_name = "bigscience/bloom-7b1"
# model_name = "facebook/opt-13b"
model_name = "facebook/opt-6.7b"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, 
                                             device_map="auto")

#for Flan-T5
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     quantization_config=BitsAndBytesConfig(load_in_8bit=True)
# )

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

2025-06-02 19:01:34.064836: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 19:01:34.078318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748890894.094650 3732503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748890894.099653 3732503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748890894.111907 3732503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
prompt = (
    "Explain how black holes form and what happens inside them. "
    "Be detailed and use analogies a student could understand."
)
prompt = (
    "Explain the history of the Roman Empire in great detail, starting from the founding of Rome through the Republic, "
    "the rise of Julius Caesar, the reign of Augustus, and the eventual decline of the Empire. "
    "Include political, military, and cultural developments along the way." * 100
)

dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# Show 8 long prompts (instruction + context if available)
batch_size = 16
batch_prompts = []
for i in range(len(dataset)):
    example = dataset[i]
    prompt = example["instruction"]
    if example.get("context"):
        prompt += "\n" + example["context"]
    # Filter long prompts
    # if len(prompt.split()) > 100:
    batch_prompts.append(prompt)
    if len(batch_prompts) == batch_size:
        break


In [4]:
# Tokenization
# inputs = tokenizer(batch_prompts, return_tensors="pt", truncation=True, max_length=2048,padding=True)
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048,padding=True)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# --------------------------
# Prefill (first forward pass)
# --------------------------
start = time.time()
with torch.no_grad():
    prefill_outputs = model(**inputs, use_cache=True)
prefill_time = time.time() - start
print(f"[Prefill] {prefill_time:.3f} sec")

# --------------------------
# Decode (token-by-token)
# --------------------------
past_key_values = prefill_outputs.past_key_values
generated = inputs["input_ids"]
max_new_tokens = 512

# decode_start = time.time()
# for _ in range(max_new_tokens):
#     with torch.no_grad():
#         outputs = model(input_ids=generated[:, -1:], past_key_values=past_key_values, use_cache=True)
#         next_token = outputs.logits[:, -1].argmax(dim=-1, keepdim=True)
#         past_key_values = outputs.past_key_values
#         generated = torch.cat([generated, next_token], dim=-1)
# torch.cuda.synchronize()
# decode_time = time.time() - decode_start
# print(f"[Decode] {decode_time:.3f} sec")

[Prefill] 0.196 sec
[Decode] 8.677 sec
